In [12]:
def extEucl(m, n):
    """
    Возвращает тройку (d,u,v) таких, что d — наибольший общий делитель пары (m,n) и d=um+vn
    """
    (a, b) = (m, n)
    u1 = 1; v1 = 0
    u2 = 0; v2 = 1

    while b != 0:
        assert (a == u1*m + v1*n and b == u2*m + v2*n)

        q = a // b; r = a % b
        assert (a == q*b + r)

        (a, b) = (b, r)
        (u1, u2) = (u2, u1 - q*u2)
        (v1, v2) = (v2, v1 - q*v2)

    if a >= 0:
        return (a, u1, v1)
    else:
        return (-a, -u1, -v1)
    
def invmod(x, m):
    """
    Возвращает обратный к x элемент, если x обратим (т.е. целые числа x,m взаимно простые),
    иначе - исключение типа ValueError
    """
    assert(m != 0)
    (d, u, v) = extEucl(m, x)
    if d == 1:
        return v%m
    else:
        raise ValueError("Element is not invertible modulo m")
        
def calc_comparison(h,m,a=1):
    """
    Решение сравнения: a*x≡h(mod m)
    """
    q=[]
    (a, b) = (m, a)
    c=m
    while b != 0:
        (a, b) = (b, a%b)     
        q.append((c-b)//a)
        c=a
        
    P=[0 for i in range(len(q))]
    for i in range(len(q)):
        if i==0:
            P[0]=q[0]
        elif i==1:
            P[1]=q[1]*P[0]+1
        else:
            P[i]=q[i]*P[i-1]+P[i-2]
            
    return (((-1)**(len(q)-1))*P[-2]*h)%m

## Алиса выбирает открытые и закрытые параметры

* q - большое положительное целое число
* f - целое число, удовлетворяющее условию: $$f<\sqrt{\frac{q}{2}}$$
* g - целое число, удовлетворяющее условию: $$\sqrt{\frac{q}{4}}<g<\sqrt{\frac{q}{2}}$$

Так же, f и g подбираются таким образом, что:
$$(f,qg) = 1$$

In [13]:
q = 122430513841
f = 231231
g = 195698

Вычисляем $$f^{-1}$$ по модулю **q**

In [21]:
f_i=invmod(f,q)
assert f_i==49194372303

Вычисляем $$h \equiv f^{−1}g \pmod{q}$$

In [15]:
h=(g*f_i)%q
assert h==39245579300

##### Открытый ключ Алисы - это пара (q, h). Закрытый - (f, g)

## Очередь Боба

* m - исходное сообщение в виде числа такое, что: $$0<m<\sqrt{\frac{q}{4}}$$
* r - случайное целое число, такое что: $$0<r<\sqrt{\frac{q}{2}}$$

In [16]:
m = 123456
r = 101010

Шифруем исходное сообщение с помошью открытого ключа Алисы: $$e \equiv rh+m \pmod{q}$$ причём $$0 < e < q$$

In [17]:
e=(r*h+m)%q
assert e==18357558717

##### Передаём **e** по каналу связи

## Алиса принимает сообщение

Вычисляем $$a \equiv fe\pmod{q}$$ причём $$0 < a < q$$ 

In [18]:
a=(f*e)%q
assert a==48314309316

Вычисляем $$f^{-1}$$ по модулю **g**

In [22]:
f_i=invmod(f,g)
assert f_i==193495

Вычисляем $$b \equiv f^{-1}a\pmod{g}$$ причём $$0 < b < g$$

In [20]:
d=(f_i*a)%g
assert d==123456

##### Готово! Мы получили исходное сообщение.

## Вмешивается Ева